**Get data**

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')


In [ ]:
# cd drive/My\ Drive

**Import necessary libraries**

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import random
from sklearn import metrics
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, InputLayer
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
import matplotlib.pyplot as plt

**Upsample/duplicate counterexamples**

*Technique 1*

In [ ]:
def upsample(eps_df, upsample_to = 10):

  cnt = 0
  upsample_lst = []
  
  # Get feature data
  counter_x = eps_df.iloc[:,2:27].to_numpy()
  counter_y = eps_df.iloc[:,27:].to_numpy()
  print("Number of counter examples:", counter_x.shape[0])

  # Extract epsilons from counterexample data
  eps = eps_df.to_numpy()[:,1]

  # Upsample counterexamples 
  # Technique 1: Upsample counterexamples by choosing a random number between 
  # [epsilon, epsilon + 0.001] and adding and subtracting it to one counterexample.
  # Repeat this procedure n times to generate 2n more examples per counterexample.

  for i,j in zip(counter_x, counter_y):
    
    # Get epsilon for the counterexample
    epsilon = eps[cnt]

    # Upsampling for one counterexample
    for n in range(upsample_to):

      # Generate random number
      r = random.uniform(epsilon, epsilon+0.001)
      # Add random number to counterexample
      i_new_high = i + r
      # Subtract random number from counterexample
      i_new_low = i - r
      # Append the newly formed feature vectors with the target vector
      ij_new_high = np.hstack((i_new_high, j))
      ij_new_low = np.hstack((i_new_low, j))
    
      upsample_lst.append(ij_new_high)
      upsample_lst.append(ij_new_low)

    cnt += 1

  print("Number of upsampled counterexamples:",len(upsample_lst))
  return np.asarray(upsample_lst)

*Technique 2*

In [ ]:
def upsample_duplicate(eps_df, upsample_to = 10):

  cnt = 0
  upsample_lst = []
  
  # Get feature data
  counter_x = eps_df.iloc[:,2:27].to_numpy()
  counter_y = eps_df.iloc[:,27:].to_numpy()
  print("Number of counter examples:", counter_x.shape[0])

  # Extract epsilons from counterexample data
  eps = eps_df.to_numpy()[:,1]

  # Upsample counterexamples 
  # Technique 2: Duplicate every counterexample n times to generate n more examples.

  for i,j in zip(counter_x, counter_y):

    # Get epsilon for the counterexample
    epsilon = eps[cnt]

    # Upsampling for one counterexample
    for n in range(upsample_to):
      ij_new = np.hstack((i, j))
      upsample_lst.append(ij_new)
    cnt += 1

  print("Number of upsampled counterexamples:",len(upsample_lst))
  return np.asarray(upsample_lst)

**Train function**

In [ ]:
def train(X_train, X_test, y_train, y_test, epochs = 30, batch_size = 16):


  # Define model architecture
  model = Sequential()
  model.add(InputLayer(input_shape=(X_train.shape[1],)))
  model.add(Dense(23, activation='relu', kernel_initializer='he_normal'))
  model.add(Dense(18, activation='relu'))
  model.add(Dense(11, activation='relu'))
  model.add(Dense(5, activation='softmax')) # logits layer

  # Define optimizer
  optimizer = 'adam'
  # Define crtierion and compile
  model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

  # Save checkpoint
  checkpoint_path = "cou_ex_model_duplicate.ckpt"
  checkpoint_dir = os.path.dirname(checkpoint_path)

  # Create a callback that saves the best model based on validation accuracy
  cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                  save_best_only=True, monitor='val_accuracy', save_weights_only = False,
                                                  verbose=1)
  
  # Train
  history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=epochs,
                    batch_size=batch_size,
                    callbacks=[cp_callback])
  
  # Save model in .pb and .h5 format
  model.save('./duplicate_counterex_model')
  model.save('./duplicate_counterex_model.h5', save_format='h5')
  
  return history



**Plot loss and accuracy curves**

In [ ]:
def plot_acc(history):
  # summarize history for accuracy

  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()

In [ ]:
def plot_loss(history):
  # summarize history for loss
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()

**Main function | Read data, upsample counterexample data, train and evaluate the model**

In [ ]:
if __name__ == "__main__":

  # Read data
  print("~ Reading data...")
  df_counter_784 = pd.read_csv("counterexamples_adversarial.csv")
  df_counter_784_eps = pd.read_csv("epsilon_counterexamples.csv")
  df_org_trainfile = pd.read_csv("v3.2.2_train.csv")
  df_counter_2000 = pd.read_csv("counterexamples2000.csv")
  df_counter_2000_eps = pd.read_csv("epsilon_counterexamples2000.csv")
  df_counter_clustering = pd.read_csv("region_counterexamples_clustering.csv")
  df_counter_clus_eps_4000 = pd.read_csv("eps_counter_clustering4000.csv")

  print("~ Done")

  print("-"*30)
  print("~ Preprocessing...")
  
  counterex_robust = df_counter_2000_eps.iloc[:,2:].to_numpy()
  counterex_clus = df_counter_clus_eps_4000.iloc[:, 1:].to_numpy()

  # Upsample required data
  upsample_counterex_robust = upsample(df_counter_2000_eps.iloc[:,1:], upsample_to = 100)

  upsample_counterex_clus = upsample(df_counter_clus_eps_4000, upsample_to = 100)


  # Original data
  org_train_np = df_org_trainfile.iloc[:,1:].to_numpy()

  print("Original data samples", org_train_np.shape[0])
  print("Total Upsampled examples", counterex_robust.shape[0]+upsample_counterex_robust.shape[0]+upsample_counterex_clus.shape[0]+counterex_clus.shape[0])
  
  # Create a data frame of Upsampled data + original data
  df_up = pd.DataFrame(np.vstack((counterex_robust, counterex_clus, upsample_counterex_robust, upsample_counterex_clus, org_train_np)))

  # Rename the columns with feature names
  df_up.columns = ['FixationDuration', 'FixationSeq', 'FixationStart', 
                'FixationX', 'FixationY', 'GazeDirectionLeftZ', 'GazeDirectionRightZ', 
                'PupilLeft', 'PupilRight', 'InterpolatedGazeX', 'InterpolatedGazeY', 
                'AutoThrottle', 'AutoWheel', 'CurrentThrottle', 'CurrentWheel', 
                'Distance3D', 'MPH', 'ManualBrake', 'ManualThrottle', 'ManualWheel', 
                'RangeW', 'RightLaneDist', 'RightLaneType', 'LeftLaneDist', 'LeftLaneType', 
                'TOT_fast', 'TOT_med_fast', 'TOT_med', 'TOT_med_slow', 'TOT_slow']

  # Convert df to csv and save
  df_up.to_csv('counter_robust_clus_dup.csv')

  print("~ Preprocessing done")
  print("-"*30)

  print("~ Training:")
  x = df_up.iloc[:,:25] # features
  y = df_up.iloc[:,25:] # targets

  # Split data | 80% (for training)- 20% (testing)
  X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=27, shuffle=False)

  # Train
  history = train(X_train, X_test, y_train, y_test)

  print("~ Done")
  print("-"*30)

  # Plot loss and accuracy graphs
  print("~ Plotting loss and accuracy")
  plot_acc(history)
  plot_loss(history)
